Just generation with pretrained models

In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import re
import sys
import json

Using TensorFlow backend.


In [3]:
def readfile(fname):
    with open(fname, 'r') as f:
        return f.read().lower()

def preprocess(poem):
    # left only words
    poem = re.sub('[^!а-яіїєА-ЯІЇЄ\s\,\.\-\—\:\n\!\(\)\?’]', ' ', poem)
    return poem.replace('\t', '\n')

def readjson(fname):
    with open(fname, 'r') as f:
        return json.load(f)

In [6]:
folder = './data/stus/'
file = 'Stus_Vasyl.Tom_3_1.Palimpsesty.1576.ua.txt'
raw_text = preprocess(readfile(folder + file))

char_to_int = readjson('./model/4/char_to_int.json')
int_to_char = readjson('./model/4/int_to_char.json')
int_to_char = {int(k):v for k,v in int_to_char.items()}

n_chars = len(raw_text)
n_vocab = len(char_to_int)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [2]:
!ls ./model/4

char_to_int.json		    weights-improvement-23-2.1634.hdf5
int_to_char.json		    weights-improvement-24-2.1497.hdf5
weights-improvement-00-3.2531.hdf5  weights-improvement-25-2.1422.hdf5
weights-improvement-01-3.1677.hdf5  weights-improvement-26-2.1302.hdf5
weights-improvement-02-3.0998.hdf5  weights-improvement-27-2.1169.hdf5
weights-improvement-03-2.9913.hdf5  weights-improvement-28-2.1146.hdf5
weights-improvement-04-2.8765.hdf5  weights-improvement-29-2.1039.hdf5
weights-improvement-05-2.7817.hdf5  weights-improvement-30-2.0911.hdf5
weights-improvement-06-2.7056.hdf5  weights-improvement-31-2.0854.hdf5
weights-improvement-07-2.6393.hdf5  weights-improvement-32-2.0796.hdf5
weights-improvement-08-2.5820.hdf5  weights-improvement-33-2.0724.hdf5
weights-improvement-09-2.5330.hdf5  weights-improvement-34-2.0577.hdf5
weights-improvement-10-2.4907.hdf5  weights-improvement-35-2.0567.hdf5
weights-improvement-11-2.4481.hdf5  weights-improvement-36-2.0556.hdf5
weights-improvement-12-2.4128.hdf5

In [8]:
model_file = 'model/4/weights-improvement-49-1.9976.hdf5'
model = Sequential()
model.add(LSTM(256, input_shape=(100, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(44, activation='softmax'))
model.load_weights(model_file)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"{}\"".format(''.join([int_to_char[value] for value in pattern])))
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

Seed:
"емно і глибоко.
так мудро нас страждання піднесло
понад плавбою і понад собою,
пускай на воду зламан"
а віть.
і в соло колонобоять в солодих,
толь даступаєи на столі солози
і сонце замає у легання
і від тебе не знаючись — на стрімуй
і не віде в сонце засливі,
і від тобі не віде в неме.
і світ — на солодитий кров.
аа та в сені за мертві  огорожий.
так — ти в світ замалах — на віловом
 стовасти те порожне
і від с і на віддайтя —
лодий тонкостині  а відктить
і в немов за порою. порного ж нами,
а ти в нема стало навіли світло
і високи світ — на солов’ях спогад.
так — ти в світ замалах — на віловом

скала ворони на мене від авлять
і в немов порожнечу від тебе
виз серевий світ — на стоію тінь.
так — ти в світ замалах — на віловом

і соіни  дем в світі — то тик — на віда?
ти в сонце задубання віддайти.
так де тобою серце стана
і від ою і проминува ня від,
порожин срай — на віловий срок
за колом серця столі — на столі,
та в сонце засту і стогі сток.
і співаться в надій і срові с
іта
і т

In [17]:
pattern_str = "гойдається вечора зламана віть"
pattern_str = " "*(100-len(pattern_str))+pattern_str
pattern = [char_to_int[value] for value in pattern_str]
print(pattern_str)
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

                                                                      гойдається вечора зламана віть
.
і в соло коловини в світ —
але порожнечу в світі —
і відчує світа над норі най солоних
повіки спогади і срертні сподівання.
прощай, окріши немає проминіла
в солодих кров полосіння.
прощай, окрічен і крики
за край мені за мене відча.
і да віддавояю світ — на сто просторить
і не забуття, на вітрі солози
і солоді сене на віда.
тож прощайт о небо над тобою
за колоних мосни відчуи
світ світання привікті, як привікт
від сеелю і томтра на сорокка
да солодитий світ — на сто простори
порать на колов’у і кроки
прощай, окран і полосі — на стила
і світло заступали монця,
скерти в громинути в стрінь.
так — ти в світ замалах — на віловом

скала ворони на мене від астий
і вирокий світ — на сто промторить
і не забуття полісиній
і не знаю, що в соло колових
коли на стрімує надав то  світ.
тож посостий поров над провиллні
солодаи соіпа світ — на сто простори
і солодість в світі — то тике  аверени
порож